# pickling.ipynb
This notebook was created as a central point to handle pickling of commonly accessed connection data frames. With the PAM analyses becoming larger and larger, with more and more neurons and connections fetched during each run, caching became necessary. Loading commands were copy-pasted into this Jupyter notebook to ensure consistency. 

In [9]:
import navis.interfaces.neuprint
from navis.interfaces.neuprint import fetch_adjacencies, fetch_synapse_connections
from navis.interfaces.neuprint import NeuronCriteria as NC, SynapseCriteria as SC

from dotenv import load_dotenv
import os
from utils import loadConnections
import pickle

In [10]:
load_dotenv()

## fetching Janelia client
client = navis.interfaces.neuprint.Client('https://neuprint.janelia.org/', dataset='hemibrain:v1.2.1', token=os.environ.get("JANELIA_PAT"))

In [4]:
### get PAM-PAM connections
filteredConnections = loadConnections(silent=False,cached=False)
### retrieving all connections between PAM and non-PAM neurons
filteredPAMConnections = loadConnections("^PAM.*","^.*",bidirectional=True,cached=False)

KeyboardInterrupt: 

In [8]:
# RETRIEVE ALL PAM-PAM SYNAPSES

## get PAM neurons 
neuron_criteria = NC(status='Traced', type="^PAM.*",regex=True)

## get all synapses, within and outside MB
allPAMPAMpresynapses_criteria = SC(type='pre', primary_only=True)
allPAMPAMpostsynapses_criteria = SC(type='post', primary_only=True)
allPAMPAMpresynapses = fetch_synapse_connections(neuron_criteria, neuron_criteria, allPAMPAMpresynapses_criteria,batch_size=10000)
allPAMPAMpostsynapses = fetch_synapse_connections(neuron_criteria, neuron_criteria, allPAMPAMpostsynapses_criteria,batch_size=10000)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10509 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10509 [00:00<?, ?it/s]

In [9]:
neuron_criteria = NC(status='Traced', type="^PAM.*",regex=True)
## get synapses within MB
MB_rois=["a'L(R)","aL(R)","b'L(R)","bL(R)","gL(R)","CA(L)","a'L(L)","aL(L)","b'L(L)","bL(L)","gL(L)", "CA(R)", "PED(R)"]
MBpresynapses_criteria = SC(type='pre', primary_only=True,rois=MB_rois)
MBpostsynapses_criteria = SC(type='post', primary_only=True,rois=MB_rois)
MB_PAM_PAM_presynapses = fetch_synapse_connections(neuron_criteria, neuron_criteria, MBpresynapses_criteria)
MB_PAM_PAM_postsynapses = fetch_synapse_connections(neuron_criteria, neuron_criteria, MBpostsynapses_criteria)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/9716 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/9716 [00:00<?, ?it/s]

In [10]:
## get synapses outside MB
### previously, these ROIs have been found to be the ROIs outside the MB that have PAM-PAM connections
non_MB_rois=["CRE(L)", "CRE(R)", "EB", "LAL(R)", "SIP(L)", "SIP(R)", "SLP(R)", "SMP(L)", "SMP(R)", "LAL(L)"]
nonMBpresynapses_criteria = SC(type='pre', primary_only=True,rois=non_MB_rois)
nonMBpostsynapses_criteria = SC(type='post', primary_only=True,rois=non_MB_rois)
nonMB_PAM_PAM_presynapses = fetch_synapse_connections(neuron_criteria, neuron_criteria, nonMBpresynapses_criteria)
nonMB_PAM_PAM_postsynapses = fetch_synapse_connections(neuron_criteria, neuron_criteria, nonMBpostsynapses_criteria)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/592 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/592 [00:00<?, ?it/s]

In [11]:
all_neurons, roi_count = neu.fetch_neurons(NC(status='Traced',type="^.*",regex=True)) 

/Users/User/CODING/Uni/2024_DANModulation/DANModulation/lib/python3.11/site-packages/neuprint/client.py:609: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neuron_props_val = df_results.iloc[0][0]


In [12]:
## fetch all synapses that involve PAM,
## postsynapses are 1:1 identical to presynapses, both have n=150049 
pam_criteria = NC(status='Traced', type="^PAM.*",regex=True)
all_criteria = NC(status='Traced', type="^.*",regex=True)

allPAMpresynapses_criteria = SC(type='pre', primary_only=True)
allPAMpostsynapses_criteria = SC(type='post', primary_only=True)
all_PAM_All_presynapses = fetch_synapse_connections(pam_criteria, all_criteria, allPAMpresynapses_criteria,batch_size=10000)
all_PAM_All_postsynapses = fetch_synapse_connections(pam_criteria, all_criteria, allPAMpostsynapses_criteria,batch_size=10000)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/150049 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/150049 [00:00<?, ?it/s]

In [13]:
rel_path="pickles/"

In [7]:
with open(rel_path+'PAM_PAM_Connections.pkl', 'wb') as file:
    pickle.dump(filteredConnections, file)

with open(rel_path+'PAM_All_Connections.pkl', 'wb') as file:
    pickle.dump(filteredPAMConnections, file)

In [14]:
with open(rel_path+'all_PAM_PAM_presynapses.pkl', 'wb') as file:
    pickle.dump(allPAMPAMpresynapses, file)
with open(rel_path+'all_PAM_PAM_postsynapses.pkl', 'wb') as file:
    pickle.dump(allPAMPAMpostsynapses, file)

with open(rel_path+'MB_PAM_PAM_presynapses.pkl', 'wb') as file:
    pickle.dump(MB_PAM_PAM_presynapses, file)
with open(rel_path+'MB_PAM_PAM_postsynapses.pkl', 'wb') as file:
    pickle.dump(MB_PAM_PAM_postsynapses, file)

with open(rel_path+'nonMB_PAM_PAM_presynapses.pkl', 'wb') as file:
    pickle.dump(nonMB_PAM_PAM_presynapses, file)
with open(rel_path+'nonMB_PAM_PAM_postsynapses.pkl', 'wb') as file:
    pickle.dump(nonMB_PAM_PAM_postsynapses, file)

with open(rel_path+'all_neurons.pkl', 'wb') as file:
    pickle.dump(all_neurons, file)

with open(rel_path+'all_PAM_All_presynapses.pkl', 'wb') as file:
    pickle.dump(all_PAM_All_presynapses, file)
with open(rel_path+'all_PAM_All_postsynapses.pkl', 'wb') as file:
    pickle.dump(all_PAM_All_postsynapses, file)